In [1]:
# https://discuss.pytorch.org/t/call-backward-on-function-inside-a-backpropagation-step/3793
# https://discuss.pytorch.org/t/implementing-a-custom-convolution-using-conv2d-input-and-conv2d-weight/18556
# https://discuss.pytorch.org/t/implementing-a-custom-convolution-using-conv2d-input-and-conv2d-weight/18556/21

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import numpy as np

In [3]:

class Conv2dFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, weight, bias=None, stride=1, padding=1, dilation=1, groups=1):
        # Save arguments to context to use on backward
        # WARNING : if stride, padding, dilation etc is array, this will not work properly!!!!
#         print('stride', stride)
        if weight.shape[2] == 1 :
            padding = 0
        elif weight.shape[2] == 5 :
            padding = 2
        elif weight.shape[2] == 7 :
            padding = 3
        confs = torch.from_numpy(np.array([stride, padding, dilation, groups]))
        out = F.conv2d(input, weight, bias=bias, stride=stride, padding=padding, dilation=dilation, groups=groups)
        ctx.save_for_backward(input, out, weight, bias, confs)

        # Compute Convolution
        return out
    
    @staticmethod
    def backward(ctx, grad_output):
        # Load saved tensors
        input, out, weight, bias, confs = ctx.saved_variables
        confs = confs.numpy()
        stride, padding, dilation, groups= confs[0], confs[1], confs[2], confs[3]

        # Calculate Gradient
        grad_input = grad_weight = grad_bias = None
#         print('grad_output', grad_output.shape)
#         print('out', out.shape)
#         print(out[0][0][0])
#         print(out.shape)
#         print(torch.max(out))
#         print(torch.min(out))
# 2*torch.sigmoid(3*x/torch.max(x))
        grad_output = grad_output * 2*torch.sigmoid(out)
        if ctx.needs_input_grad[0]:
            grad_input = torch.nn.grad.conv2d_input(input.shape, weight, grad_output, stride, padding, dilation, groups)
            
        if ctx.needs_input_grad[1]:
            grad_weight = torch.nn.grad.conv2d_weight(input, weight.shape, grad_output, stride, padding, dilation, groups)
                
        # WARNING : Bias maybe buggy, remove if it is buggy
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0).squeeze(0)


        # WARNING : Bias maybe buggy, remove if it is buggy
        if bias is not None:
            return grad_input, grad_weight, grad_bias, None, None, None, None
        else:
            return grad_input, grad_weight, None, None, None, None, None

In [4]:
from torch.autograd import gradcheck
conv = Conv2dFunction.apply
# gradcheck takes a tuple of tensors as input, check if your gradient
# evaluated with these tensors are close enough to numerical
# approximations and returns True if they all verify this condition.
# input = (torch.randn(20,20,dtype=torch.double,requires_grad=True), torch.randn(30,20,dtype=torch.double,requires_grad=True))
# test = gradcheck(linear, input, eps=1e-6, atol=1e-4)
# print(test)


In [5]:
# device = torch.device("cpu")
device = torch.device("cuda")
dtype = torch.float
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='../data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='../data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [6]:
# N, C_in, C_out, K_size = batch_size, 3, 12, 3
# Create random Tensors for weights.
conw1 = torch.randn(8,3,5,5, device=device, dtype=dtype, requires_grad=True)
conw2 = torch.randn(32,8,3,3, device=device, dtype=dtype, requires_grad=True)
conw3 = torch.randn(128,32,3,3, device=device, dtype=dtype, requires_grad=True)
conw4 = torch.randn(128,128,3,3, device=device, dtype=dtype, requires_grad=True)
conw5 = torch.randn(10,128,1,1, device=device, dtype=dtype, requires_grad=True)

conw1 = torch.nn.init.xavier_uniform_(conw1, gain=1.0)
conw2 = torch.nn.init.xavier_uniform_(conw2, gain=1.0)
conw3 = torch.nn.init.xavier_uniform_(conw3, gain=1.0)
conw4 = torch.nn.init.xavier_uniform_(conw4, gain=1.0)
conw5 = torch.nn.init.xavier_uniform_(conw5, gain=1.0)

# print(conw1[0][0])
# print(torch.nn.init.xavier_uniform_(conw1, gain=1.0)[0][0])
# print(conw2)

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = Conv2dFunction.apply
        self.conv2 = Conv2dFunction.apply
        self.conv3 = Conv2dFunction.apply
        self.conv4 = Conv2dFunction.apply
        self.conv5 = Conv2dFunction.apply
        self.avgpool = torch.nn.AvgPool2d((2,2) ,stride=(2,2))
        self.maxpool = torch.nn.MaxPool2d((2,2), stride=(2,2))
        self.linear = torch.nn.Linear(128, 10)
        self.act = torch.nn.ReLU()

    def forward(self, x, w1, w2, w3, w4, w5):
#         x = self.conv1(x, w1)
#         x = self.act(x)
#         x = self.conv2(x, w2)
#         x = self.maxpool(x)
#         x = self.conv3(x, w3)
#         x = self.act(x)
#         x = self.conv4(x, w4)
#         x = self.avgpool(x)
#         x = torch.squeeze(x)
#         x = self.linear(x)
#         x = torch.nn.Softmax(dim=1)(x)

        x = self.act(self.conv1(x, w1))
        x = torch.nn.BatchNorm2d(8).to(device)(x)
        x = self.maxpool(x)
        x = self.act(self.conv2(x, w2))
        x = torch.nn.BatchNorm2d(32).to(device)(x)
        x = self.maxpool(x)
        x = self.act(self.conv3(x, w3))
        x = torch.nn.BatchNorm2d(128).to(device)(x)
        x = self.maxpool(x)
        x = self.act(self.conv4(x, w4))
        x = torch.nn.BatchNorm2d(128).to(device)(x)
        x = self.maxpool(x)
        x = self.conv5(x, w5)
        x = self.avgpool(x)
        x = torch.squeeze(x)
#         x = self.linear(x)
        x = torch.nn.Softmax(dim=1)(x)
#         x = torch.sigmoid(x)
        
        return x

In [8]:
net = Net().to(device)
criterion = nn.CrossEntropyLoss().to(device)

In [9]:
image, label = iter(trainloader).next()
print(image.shape)

torch.Size([64, 3, 32, 32])


In [10]:
image, labels = iter(trainloader).next()
outputs = net(image.to(device), conw1, conw2, conw3, conw4, conw5).to(device)
print(outputs.shape)
# print(outputs.sum(dim=1))
# print(outputs)
loss = criterion(outputs, labels.to(device))
loss.backward()
# print(torch.nn.Softmax(dim=1)(outputs).sum(dim=1))
# outputs


torch.Size([64, 10])


/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'


In [11]:
# conw5.grad

In [12]:
def test (model, w1, w2, w3, w4, w5) :
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data

            outputs = model(inputs.to(device), w1, w2, w3, w4, w5)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.to(device)).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))

In [13]:
lr_list = [0.05, 0.01, 0.01, 0.002, 0.001, 0.001, 0.0002, 0.0001, 0.00002, 0.00001]
# lr_list = [0.05] * 10
criterion = nn.CrossEntropyLoss().to(device)
for epoch in range(10) :    
    running_loss = 0.0
    learning_rate = lr_list[epoch]
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        outputs = net(inputs.to(device), conw1, conw2, conw3, conw4, conw5)
#         print(outputs.shape)
#         print(labels.shape)
        loss = criterion(outputs, labels.to(device))
#         print(loss)
        loss.backward()    

        # print statistics
        running_loss += loss.item()
        if i % 200 == 0:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0 
#             print(conw1[0][0][0])
#             print(conw5[0][0][0])
#             print(conw4.grad)
            
        with torch.no_grad():
            # Update weights using gradient descent
            conw1 -= learning_rate * conw1.grad
            conw2 -= learning_rate * conw2.grad
            conw3 -= learning_rate * conw3.grad
            conw4 -= learning_rate * conw4.grad
            conw5 -= learning_rate * conw5.grad

            # Manually zero the gradients after running the backward pass
            conw1.grad.zero_()
            conw2.grad.zero_()   
            conw3.grad.zero_()
            conw4.grad.zero_()       
            conw5.grad.zero_()                   
            
    test(net, conw1, conw2, conw3, conw4, conw5)

[1,     1] loss: 0.012


/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'


[1,   201] loss: 2.108
[1,   401] loss: 2.014
[1,   601] loss: 1.973
Accuracy of the network on the 10000 test images: 49 %
[2,     1] loss: 0.009
[2,   201] loss: 1.907
[2,   401] loss: 1.896
[2,   601] loss: 1.894
Accuracy of the network on the 10000 test images: 56 %
[3,     1] loss: 0.009
[3,   201] loss: 1.871
[3,   401] loss: 1.864
[3,   601] loss: 1.868
Accuracy of the network on the 10000 test images: 59 %
[4,     1] loss: 0.009
[4,   201] loss: 1.850
[4,   401] loss: 1.840
[4,   601] loss: 1.840
Accuracy of the network on the 10000 test images: 60 %
[5,     1] loss: 0.009
[5,   201] loss: 1.833
[5,   401] loss: 1.838
[5,   601] loss: 1.835
Accuracy of the network on the 10000 test images: 60 %
[6,     1] loss: 0.009
[6,   201] loss: 1.830
[6,   401] loss: 1.834
[6,   601] loss: 1.833
Accuracy of the network on the 10000 test images: 60 %
[7,     1] loss: 0.009
[7,   201] loss: 1.836
[7,   401] loss: 1.821
[7,   601] loss: 1.829
Accuracy of the network on the 10000 test images:

In [14]:
# filters = torch.randn(12, 3, 3, 3)
# inputs = torch.randn(2, 3, 32, 32)
# F.conv2d(inputs, filters, padding=1).shape

In [15]:
import torch
x = torch.randn(4,6)
print(x.shape)
# torch.nn.Softmax(dim=1)(x)
# x.clamp(min=0)
x

torch.Size([4, 6])


tensor([[ 0.3808, -0.5430,  0.2496,  0.0992,  0.8057,  0.1304],
        [ 1.6136, -0.6286, -0.7112,  0.6455, -0.0244,  0.1755],
        [ 0.5516, -0.6623,  0.2483,  0.4446,  0.1969, -0.2364],
        [ 0.9153, -0.2632,  0.1100,  0.2699, -0.6105,  0.0084]])

In [16]:
3*x/torch.max(x)

tensor([[ 0.7080, -1.0095,  0.4640,  0.1845,  1.4979,  0.2425],
        [ 3.0000, -1.1687, -1.3222,  1.2001, -0.0453,  0.3263],
        [ 1.0255, -1.2313,  0.4617,  0.8266,  0.3661, -0.4396],
        [ 1.7018, -0.4893,  0.2045,  0.5018, -1.1350,  0.0156]])

In [17]:
2*torch.sigmoid(3*x/torch.max(x))

tensor([[1.3399, 0.5341, 1.2279, 1.0920, 1.6345, 1.1207],
        [1.9051, 0.4742, 0.4209, 1.5371, 0.9773, 1.1617],
        [1.4721, 0.4519, 1.2268, 1.3913, 1.1810, 0.7837],
        [1.6915, 0.7601, 1.1019, 1.2458, 0.4865, 1.0078]])